In [ ]:
# Cell 7 (updated): SHAP summary plot with explicit RNG when supported, otherwise suppress FutureWarning
import numpy as np
import shap
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import inspect

# Preconditions (must exist in the session):
# - sm                : normalized SHAP array shape (n_samples, n_features)
# - X_mis_trans       : transformed numeric array for misclassified examples
# - X_train_trans     : transformed numeric array for training set
# - trans_feature_names: list of transformed feature names

plot_X = X_mis_trans
feature_names = trans_feature_names if 'trans_feature_names' in globals() else [f"f{i}" for i in range(sm.shape[1])]

# Create a new Generator for reproducible randomness
rng = np.random.default_rng(0)

# Check whether shap.summary_plot accepts an rng argument
accepts_rng = False
try:
    sig = inspect.signature(shap.summary_plot)
    accepts_rng = 'rng' in sig.parameters
except Exception:
    # If signature inspection fails, we'll attempt to call with rng and handle TypeError below
    accepts_rng = False

if accepts_rng:
    # Preferred path: pass explicit rng to silence the FutureWarning and make randomness explicit
    shap.summary_plot(sm, plot_X, feature_names=feature_names, show=True, rng=rng)
else:
    # Fallback: attempt to call with rng (older SHAP may raise TypeError)
    try:
        shap.summary_plot(sm, plot_X, feature_names=feature_names, show=True, rng=rng)
    except TypeError:
        # Older SHAP uses np.random.seed internally; seed legacy RNG and suppress FutureWarning
        np.random.seed(0)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            ##Learners to type
            

# If the summary plot fails for other reasons, provide a static fallback
try:
    plt.gcf().canvas.draw_idle()
except Exception:
    mean_abs = pd.Series(np.abs(sm).mean(axis=0), index=feature_names).sort_values(ascending=True)
    n_display = min(30, len(mean_abs))
    mean_abs.tail(n_display).plot(kind="barh", figsize=(10, max(4, 0.25 * n_display)))
    plt.title("Mean absolute SHAP (fallback)")
    plt.xlabel("mean |SHAP value|")
    plt.tight_layout()
    plt.show()
